In [1]:
# Load libraries

import pandas as pd
import numpy as np

#import missingno as msno
import statsmodels.api as sm

#from fbprophet import Prophet

#import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from pylab import rcParams
rcParams['figure.figsize'] = 15, 12

# Allows to display all of the outputs of a cell
from IPython.display import display

# Set float data type format
pd.options.display.float_format = '{:,.2f}'.format

# Set the maximum number of row to be displayed
pd.options.display.max_rows = 999

# Set global visualisation settings
plt.rc('font', size=14)  

# Load Data

In [2]:
df = pd.read_csv('./housing_in_london_monthly_variables.csv')
df_1 = pd.read_csv('./housing_in_london_yearly_variables.csv')
print('The Month dataset contains {} rows and {} columns.'.format(df.shape[0], df.shape[1]))
print ('This Year dataset contains {} rows and {} columns.'.format(df_1.shape[0], df_1.shape[1]))
display(df.head())
display(df_1.head())

FileNotFoundError: [Errno 2] No such file or directory: './housing_in_london_monthly_variables.csv'

# Data Info

In [ ]:
df_1.groupby("area").count()

In [ ]:
df.info()
print('          ')
df_1.info()

# Missing Data

In [ ]:
#Monthly Missing Data
null_df = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending = False)*100

null_df = pd.concat([null_df, percent], axis = 1, keys = ['Counts', '% Missing'])
print ('Monthly Missing Data: ')
null_df.head()

In [ ]:
#Yearly Missing Data
null_df_1 = df_1.isnull().sum().sort_values(ascending = False)
percent = (df_1.isnull().sum()/df_1.isnull().count()).sort_values(ascending = False)*100

null_df_1 = pd.concat([null_df_1, percent], axis = 1, keys = ['Counts', '% Missing'])
print ('Yearly Missing Data: ')
null_df_1.head()

# Missing Data Processing

In [ ]:
#For monthly, it has a very large missing ratio on "no_of_crimes" so we decide to delete this column
df.drop('no_of_crimes', axis = 1, inplace = True)
#For monthly, fill NaN values with the mean of that particular area
df['houses_sold'].fillna(df.groupby('area')['houses_sold'].transform('mean'), inplace = True)
display(df.head())

In [ ]:
#For yearly, it has a very large missing ratio on "life_satisfaction" so we decide to delete this column
df_1.drop('life_satisfaction', axis = 1, inplace = True)
missing_part = ['no_of_houses','area_size','recycling_pct','number_of_jobs']
df_1['no_of_houses'].fillna(df_1.groupby('area')['no_of_houses'].transform('mean'), inplace = True)
df_1['area_size'].fillna(df_1.groupby('area')['area_size'].transform('mean'), inplace = True)
df_1['number_of_jobs'].fillna(df_1.groupby('area')['number_of_jobs'].transform('mean'), inplace = True)
display(df_1.head())

In [ ]:
# Set date as index for easier manipulation
df = df.set_index(pd.to_datetime(df['date']))
df_1 = df_1.set_index(pd.to_datetime(df_1['date']))

del df['date']
del df_1['date']

df_1['mean_salary'] = df_1['mean_salary'].replace('-', np.NaN)
df_1['mean_salary'] = df_1['mean_salary'].replace('#', np.NaN)
df_1['mean_salary'] = df_1['mean_salary'].astype(float)

# Create dataset cuts
prices_london = df[df['borough_flag'] == 1]
prices_england = df[df['area'] == 'england']
prices_north_east = df[df['area'] == 'north east']

# Calcualte mean prices for the different cuts of data
london_mean_price = prices_london.groupby('date')['average_price'].mean()
england_mean_price = prices_england.groupby('date')['average_price'].mean()
north_east_mean_price = prices_north_east.groupby('date')['average_price'].mean()


display(df.head())
display(df_1.head())
print('Processing Complete')


# data display

In [ ]:
prices_london.head()

In [ ]:
# areas in london
prices_london.area.unique()

In [ ]:
prices_london.index

In [ ]:
fig = px.line(prices_london, x=prices_london.index, y="average_price", color='area')

fig.update_layout(
    template='gridon',
    title='Average Monthly London House Price by Area',
    xaxis_title='Year',
    yaxis_title='Price (£)' 
)

fig.show()